In [9]:
import pandas as pd
import numpy as np

In [37]:
# Example
n = 10

ls = []
for index in range(n):
    hours = np.random.choice(np.arange(24), size=n)
    minutes = np.random.choice(np.arange(60), size = n )
    seconds = np.random.choice(np.arange(60), size = n)
    
    idx = [pd.datetime(2011, 2, 7, h,m,s) for h,m,s in zip(hours,minutes, seconds)]
    values = np.random.choice([1,2,3], size=n)
    
    ls.append(pd.Series(values, index=pd.Index(idx), name="purchases"))

    
df = pd.concat(ls, axis=0, keys=["cat"+str(i) for i in range(n)])
df.index.names = ["cat", "time"]
df = df.reset_index()

In [38]:
df.head()

,cat,time,purchases
0,cat0,2011-02-07 15:20:44,3
1,cat0,2011-02-07 08:42:11,1
2,cat0,2011-02-07 08:24:05,1
3,cat0,2011-02-07 18:17:30,1
4,cat0,2011-02-07 10:23:45,1


In [66]:
t = df.sort_values(["cat", "time"])
cat = t.cat.unique()
t = t[t.cat.isin(cat[:100000])].reset_index(drop=True)

In [93]:
%%time
##### Rolling window with my functions and groupby - fastest
ls1 = t.groupby("cat").apply(lambda df: (df.cat.unique()[0], rolling_sum(df.purchases.values, df.time.values, dt=np.timedelta64(1, 'm'))))

Wall time: 11 ms


In [98]:
%%time
##### Rolling window with forward fill and groupby loop - less slow
t2 = t.set_index(["time"])
ls2 = t2.groupby("cat")["purchases"].apply(lambda df: (
        df.resample("1s")
        .ffill(limit=0)
        .rolling(window=60, min_periods=1)
        .agg(np.nansum)
        ))

Wall time: 215 ms


In [99]:
ls1[0][1]

[2, 2, 1, 1, 1, 3, 1, 1, 1, 1]

In [91]:
ls2.dropna().shape

(1090,)

In [100]:
ls2.dropna().head()

cat   time               
cat0  2011-02-07 00:50:17    2.0
      2011-02-07 00:50:18    2.0
      2011-02-07 00:50:19    2.0
      2011-02-07 00:50:20    2.0
      2011-02-07 00:50:21    2.0
dtype: float64

In [14]:
# My cleanest code

def window_index(times, dt):
    """
    Returns two lists - forward window f and backward window p defined as follows:
    For each index i in times: 
        f[i] is the smallest index j <= i such that (times[i]-times[j]) <= dt
        p[i] is the largest index k >= i such that (times[k]-times[i]) <= dt
    
    Parameters
    ------------------------
    times: list of np.datetime64  of non decreasing times
    dt: np.datetime64
    
    Returns
    -------------------------
    f: list of ints - forward window for each index as above
    p: list of ints - past window for each index as above
    """
    l = len(times)
    f = np.arange(0, l)
    p = np.arange(0, l)
    
    i,j = 0,0
    while (j < l):
        diff = times[j] - times[i]
        if (diff <= dt):
            p[j] = i
            j = j+1
        elif (diff > dt):
            f[i] = (j-1)
            i = i+1
    return p,f

 
def sum_future(values, jump):
    """ Given list of forward window compute sum corresponding to that window for each index """
    totals = [np.sum(values[0:jump[0]+1])]
    for i in range(1, len(values)):
        totals.append(totals[i-1] - values[i-1] + np.sum(values[jump[i-1]+1:jump[i]+1]))
    return totals

   
def sum_past(values, jump):
    """ Given list of backward windows compute sum corresponding to that window for each index """
    l = len(values)-1
    return sum_future(values[::-1], [l-j for j in jump[::-1]])[::-1]


# From these three functions we can do lots of things including compute the rolling mean
def rolling_mean_window(values, times, dt):
    p,f = window_index(times, dt)
    return (np.array(sum_future(values,f)) + np.array(sum_past(values, p)) - np.array(values))/(f-p+1)

def rolling_sum(values, times, dt):
    p,f = window_index(times, dt)
    return sum_future(values,f)

In [ ]:
%%time
##### Rolling window with my functions and for loops - slow

t1 = trans.reset_index().sort_values(["AdvisorID", "Date"]).set_index("Date")

advisorID = t1.AdvisorID.unique()
ls = []
for a in advisorID[:1000]:
    df = t1[t1.AdvisorID == a].Transactions
    ls.append((
      rolling_sum(df.values.ravel(), df.index.values.ravel(), dt=np.timedelta64(1, 'm')),
        a))

In [ ]:
##### Rolling window with forward fill and for loop - SLOW
t1 = trans.reset_index().sort_values(["AdvisorID", "Date"]).set_index("Date")

advisorID = t1.AdvisorID.unique()
ls = []
for a in advisorID[:1000]:
    ls.append((
        (t1[t1.AdvisorID == a].Transactions)
     .resample("1M")
     .ffill(limit=0)
     .rolling(window=12, min_periods=1)
     .agg(np.nansum),
        a)
     )
    
    
s = (l[0] for l in ls)
n = (l[1] for l in ls)


df = pd.concat(s, keys=n, axis=0,).reset_index(level=1)
df.index.name = "AdvisorID"
df = df.reset_index()
df = pd.merge(t1.reset_index(), df, how="outer", on=["AdvisorID", "Date"], suffixes=["", "_agg"])